In [18]:
!pip install streamlit opencv-python-headless scikit-learn pyngrok
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.128.64:8501

  Stopping...
  Stopping...


In [24]:
%%writefile app.py

import streamlit as st
import cv2
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
import io
import base64

# Convert RGB to HEX
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

# Extract dominant colors from image
def get_colors(image, num_colors):
    image = image.resize((100, 100))  # Resize for performance
    img_array = np.array(image)
    img_array = img_array.reshape((-1, 3))

    kmeans = KMeans(n_clusters=num_colors, n_init=10, random_state=42)
    kmeans.fit(img_array)
    colors = kmeans.cluster_centers_.astype(int)

    return [rgb_to_hex(c) for c in colors]

# Generate color palette image
def create_palette_image(hex_colors):
    swatch_width = 60
    palette = Image.new("RGB", (swatch_width * len(hex_colors), 60), (255, 255, 255))
    for i, hex_color in enumerate(hex_colors):
        color = tuple(int(hex_color.lstrip("#")[j:j+2], 16) for j in (0, 2, 4))
        swatch = Image.new("RGB", (swatch_width, 60), color)
        palette.paste(swatch, (i * swatch_width, 0))
    return palette

# Create download link for palette image
def get_download_link(img, filename="palette.png"):
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    b64 = base64.b64encode(buf.getvalue()).decode()
    href = f'<a href="data:image/png;base64,{b64}" download="{filename}">📥 Download Palette</a>'
    return href

# Streamlit UI
st.set_page_config(page_title="AI Color Palette Generator", layout="centered")
st.title("🎨 AI Color Palette Generator")
st.markdown("Upload an image to generate a color palette based on the entire image.")

uploaded = st.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])

if uploaded:
    image = Image.open(uploaded).convert("RGB")
    st.image(image, caption="Uploaded Image", use_container_width=True)

    num_colors = st.slider("Number of colors", min_value=2, max_value=10, value=5)
    hex_colors = get_colors(image, num_colors)

    st.subheader("🎯 Extracted Colors (Hex)")
    cols = st.columns(len(hex_colors))
    for i, col in enumerate(cols):
        col.markdown(f"<div style='text-align:center; font-weight:bold'>{hex_colors[i]}</div>", unsafe_allow_html=True)
        col.color_picker(f"Color {i+1}", hex_colors[i], label_visibility="collapsed")

    palette_img = create_palette_image(hex_colors)
    st.image(palette_img, caption="🎨 Generated Palette", use_container_width=True)

    st.markdown(get_download_link(palette_img), unsafe_allow_html=True)


Overwriting app.py


In [25]:
from pyngrok import conf, ngrok

conf.get_default().auth_token = "2xrEFPA8892ofGRaNQaa4cPHwXz_4eyCje3Cc3M3WNMWAhMPK"


In [26]:
import os
os.system("streamlit run app.py &")
public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url)


Your Streamlit app is live at: NgrokTunnel: "https://ad3d-35-185-128-64.ngrok-free.app" -> "http://localhost:8501"
